# 10章　ヒープ

- Yusuke Matsuyama
- 7/14@book-seminar2017
- via jupyter RISE


# おきもち


1. 優先度付きキューを実装したい
1. 二分探索木を使えば優先度付きキューは作れる
    * しかし非常に難しい
1. 二分ヒープを使えば比較的簡単
    * つくろう!

# アウトライン
1. 二分ヒープの説明
    1. 完全二分木
    1. 二分ヒープ
        1. max-ヒープ
1. max-ヒープの実装
1. 優先度付きキューの実装

# 完全二分木

- すべての葉が同じ深さを持ち、全ての内部接点の子の数が2であるような二分木(a)
- 深さの差がmax1でも、ちゃんと左側から埋まってるような木も(おおよそ(?))二分木(b)

![](./imgs/CBT.png)

# 二分ヒープ
完全二分木の各ノードに割り当てられたキーが、配列の各要素に対応したデータ構造
![BH.png](./imgs/BH.png)

# indexの計算
添字iが与えられた時:

In [5]:
import math
L = 10 # length of heap

# origin 1

parent        = lambda  i : math.floor(i/2)
left_child   = lambda  i : 2 * i 
right_child = lambda  i : 2 * i + 1
parent(5),left_child(5), right_child(5)

(2, 10, 11)

# max-ヒープ条件
- 節点のキーがその親のキー以下であること(逆: minヒープ条件)
- 制約があるのは親子間のみ(兄弟間にはない)

## max-ヒープ
- max ヒープ条件を満たす二分ヒープ
![](./imgs/BH.png)


# ヒープへの要素追加
## maxheapfi(A,i) 
A[i]をmaxヒープ条件を満たすまで葉に向かって下降

1. if max(左子のキー、右子のキー) > 自分のキー　then そいつと自分を入れ替える
1. 1.を繰り返す

## max-ヒープの構築
子を持つ節点の中で添字が最大の節点sから逆順にmaxheapfi(A,s)をすればいい

![maxheapfi.png](imgs/maxheapfi.png)

# ヒープ構築の計算量
要素数をHとする。

## maxheapfi : 
    O(log 木の高さ)
## ヒープの構築:
    - 高さ1の部分木　H/2個に対してmaxheapfi
    - 高さ2の部分木　H/4個に対してmaxheapfi
    - ... 高さlog Hの部分木1つ に対してmaxheapfi
    
$$ H * \sum^{logH}_{k=1} \frac{k}{2^k} = O(H) $$


# 問題1(15分) 最大ヒープの実装(ALDS1_9_B)
与えられた配列から、max-ヒープを実装してください

## 入力
- 最初の行:配列サイズH
- H個の配列要素

```
10
4 1 3 2 16 9 10 14 8 7 
```

## 出力例
max-ヒープの節点の値を1...Hに向かって順番に空白区切で出力
```
16 14 10 8 7 9 3 2 4 1
```
## 制約
$$ 1 <= H <= 500000$$
$$ -2,000,000,000 <= 節点の値 <= 2,000,000,000 $$



In [6]:
! cat ./test.sh

#!/bin/bash

code=$1

set -eu

[ "$code" == "" ] && code="template.c"

# This is setting for C users
#compile="gcc -Wall $code"
#exec_command="./a.out"

# for example python users
compile=""
exec_command="python $code"

test_sets="A1 B1 B2"

temp=$(mktemp aojtest.XXXXXX)
trap "rm $temp" EXIT

$compile
for case in $test_sets; do
  echo [case $case]
  $exec_command < input/${case}.txt > $temp
  if diff -u output/${case}.txt $temp; then
    echo OK
  fi
done


In [7]:
# 解答1
from  math import floor
H = 0
A = []

def maxheapfi(i,A,H):
        l = 2 * (i  + 1) - 1 
        r = 2 * (i  + 1) 
        
        if l  < H and A[l] > A[i] :
            largest = l
        else:
            largest = i
        
        if r < H and A[r] > A[largest] :
            largest = r
        
        if largest != i:
            tmp = A[i]
            A[i] = A[largest]
            A[largest] = tmp
            maxheapfi(largest,A,H)

if __name__== '__main__':
    #H = int(input().strip())
    #A = map(int, input().strip().split(' '))
    lines = [x.strip() for x in open('./input/A1.txt').readlines()]
    H = int(lines[0])
    A = list(map(int, lines[1].split(' ')))
    
    for k in range(floor(H/2)-1,-1,-1):
        maxheapfi(k,A,H)
        
    print(A) 
        
    
    



[16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


# 優先度付きキュー
キーを持つ要素　の集合
## insert(S,k):
集合Sに要素kを挿入

## extractMax(S):
最大のキーを持つSの要素をSから削除し、その値を返す




# 要素の追加
1. ヒープの末尾に追加
1. ヒープ条件の確保
    - while(親より強い) : 親と入れかわる
![insertPQ.png](./imgs/insertPQ.png)

# 要素の取り出し
1. 根を記録
1. ヒープの一番末尾の要素を根に移動
1. ヒープサイズH--
1. 根からmaxheapfiを実行
1. 記憶していた旧根の値をreturn
![popPQ.png](imgs/popPQ.png)

# 問題2 : 優先度付きキューの実装(ALDS1_9_C)
## 入力:
優先度付きキューへの複数の命令
- insert k
- extract
- end

## 出力:
extract命令ごとに、優先度付きキューから取り出される値を一行ごとに出力

## 制約
$$命令の数 <= 2,000,000$$
$$0 <= k <= 2,000,000,000$$

-　入力
```入力
insert 8
insert 2
extract
insert 10
extract
imsert 11
extract
extract
```
- 出力
```出力
8
10
11
2
```

In [8]:
from math import floor

def maxheapfi(i):
    global H
    global A
    l = 2 * (i) + 1  
    r = 2 * (i +1)

    if l  < H and A[l] > A[i] :
        largest = l
    else:
        largest = i

    if r < H and A[r] > A[largest] :
        largest = r
    if largest != i:
        tmp = A[i]
        A[i] = A[largest]
        A[largest] = tmp
        maxheapfi(largest)
    
def extract():
    global H
    global A
    if H < 0 : return -1000000
    maxv = A[0]
    A[0] = A[H-1]
    H -= 1
    maxheapfi(0)
    return maxv

def increasekey(i, key):
    global H
    global A
    if key < A[i]:
        return
    A[i] = key
    while( i > 0 and A[floor((i-1)/2)] < A[i]):
        tmp = A[i]
        A[i] = A[floor((i-1)/2) ] 
        A[floor((i-1)/2)]  = tmp
        i = floor((i-1)/2)
    
        
def insert(key):
    global H
    global A
    H += 1
    if len(A) < H:
        A.append(-1000000)
    else:
        A[H-1] = -1000000
    increasekey(H-1,key)

def main():
    global H
    global A
    H = 0
    A = []
    opr = None
    count = 0
    oprators = [x.strip().split(' ') for x in open('./input/B2.txt').readlines()]
    for opr in oprators:
    #while(True):
        #opr = input().strip().split(' ')
        if opr[0] == 'insert':
            insert(int(opr[1]))
        if opr[0] == 'extract':
            print(extract())
        if opr[0] == 'end':
            break
        
main()


999999999
120
100
777
100
100
100
100
120
88
